In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from trl import SFTTrainer
from transformers import TrainingArguments
from tuning.data.train_dataset import get_train_dataset
from tuning.training.config_training import ModelLoadConfig, LoraConfig, SFTRunConfig, PTRunConfig, DPOTrainingConfig, TrainingArgumentsConfig, PassAtKConfig, sft_batch_size, effective_batch_size
from tuning.training.perplexity_callback import PerplexityStoppingCallback
from tuning.training.passk_callback import PassAtKStoppingCallback
from tuning.utils.utils import apply_chat_template, chat_template_func
import json
import sys
from datasets import load_from_disk
from typing import List, Optional, Union
from pathlib import Path
from tuning.config import DATASETS_DIR, HF_MODEL_MAP
import os
from tuning.training.config_training import DatasetConfig, SFTRunConfig
from tuning.config import MODELS_DIR
from tuning.training.sft_training import train_model_sft
from tuning.training.dpo_training import train_model_dpo
import subprocess
import importlib
import torch
import json
import wandb


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-07 15:27:47 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annota

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


In [2]:

  import importlib
  from tuning.training import passk_callback, sft_training
  importlib.reload(passk_callback)


<module 'tuning.training.passk_callback' from '/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/training/passk_callback.py'>

In [3]:
MODEL = "llama3-1B"
total_train_size = 4096  # 29980
perplexity_thresholds = [7.0,6.0, 5.75, 5.5, 5.25, 5.0, 4.75, 4.5, 4.25,4.0, 3.9, 3.8, 3.7, 3.6,3.55,3.5,3.45,3.4,3.35,3.3, 3.25, 3.2, 3.15, 3.1]
perplexity_thresholds = [6.0, 5.0]

In [4]:
dataset_config = DatasetConfig(
    dataset = "tuluif",
    dataset_type = "sft",
    train_size = total_train_size, # 29980
)

run_config = SFTRunConfig(
    dataset_config = dataset_config,
    model_name_hf = HF_MODEL_MAP[MODEL],  # Use HuggingFace model name, not local path
    model_name = MODEL,  # Base model name for output directory construction
    do_training=True,
    do_inference=False,
    do_evaluation=False,
)
passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
    target_pass_at_k=[0.1, 0.15, 0.2,0.25,0.3],
    k_values=[1],
    n_samples=1,
    num_prompts=200,
    temperature=0.7,
    strict=False,
    enabled=True,
)

lora_config = LoraConfig()
model_load_config = ModelLoadConfig()
model_load_config.max_seq_length = 4096
training_args = TrainingArgumentsConfig()
training_args.eval_steps = 1

In [ ]:
try:
    wandb.finish()
except Exception:
    pass
run = wandb.init(
    name=run_config.run_name, 
    project="tuning", 
    # reinit=True,
    # Optional: Pass config here so it's logged even if training crashes early
    config=run_config.__dict__ if hasattr(run_config, "__dict__") else {} 
)

with run:
    model, tokenizer, trainer, callbacks = train_model_sft(
        run_config = run_config,
        lora_config = lora_config,
        model_load_config = model_load_config,
        training_args = training_args,
        passk_config = passk_config
    )   

wandb: Currently logged in as: shougan (shougan-university-of-waterloo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Getting train dataset for run config: llama3-1B_sft-tuluif-4096
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-4096
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-4096
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 4096
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_se11a0hu4d0lnuekd4d6hmmb', 'prompt': 'Identify and list three groundbreaking research papers in the field of artificial intelligence from the past five years. The papers should be recognized for their innovative contributions and have received significant citations. Provide a brief summary of each paper, not exceeding 50 words per summary.', 'messages': [{'content': 'You are a helpful assi

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 4096
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 200
    })
})
{'id': 'personas_IF_se11a0hu4d0lnuekd4d6hmmb', 'prompt': 'Identify and list three groundbreaking research papers in the field of artificial intelligence from the past five years. The papers should be recognized for their innovative contributions and have received significant citations. Provide a brief summary of each paper, not exceeding 50 words per summary.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions', 'role': 'system'}, {'content': 'Identify and list three groundbreaking research papers in the field of artificial intelligence from the past five years. The papers should be recognized for their innovative contributions and have rece

Unsloth 2025.10.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


...

Adding requests:   0%|          | 0/200 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/200 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Time for prompt construction: 0.05 seconds
Time for generation: 18.59 seconds
[PassAtKCallback] Evaluating responses...

[PassAtKCallback] Step 10, Data Points 1280: pass@1=0.1300 (loose, 200 prompts)
[PassAtKCallback] Saving LoRA adapter to /tmp/tmpja6d6efe...
[PassAtKCallback] LoRA adapter saved
[PassAtKCallback] Generating 200 prompts x 1 samples (persistent, lora_id=11)...
WARNING 02-07 15:33:02 [tokenizer.py:295] No tokenizer found in /tmp/tmpja6d6efe, using base model tokenizer instead. (Exception: Unrecognized model in /tmp/tmpja6d6efe. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chamel

Adding requests:   0%|          | 0/200 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/200 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
# ppl_callback = callbacks[-1]
# metadata_file = ppl_callback.metadata_path
metadata_file = "/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models_metadata/llama3-8B_passatk-0201_2257.json"
checkpoints = []
with open(metadata_file, "r") as f:
    for line in f:
        checkpoints.append(json.loads(line))
print(checkpoints)


In [ ]:
!nvidia-smi
import gc
# del model, tokenizer, trainer, callbacks # this deletes the references to such objects
gc.collect() # then we force the GC
torch.cuda.empty_cache() # and we release the GPU CUDA cache
!nvidia-smi
# we could also use reset_cuda_context() for a clean state

In [ ]:
for checkpoint in checkpoints[2:]:
    model_name = Path(checkpoint["checkpoint_path"]).name
    data = total_train_size - checkpoint["data_points_seen"] 
    model_load_config = ModelLoadConfig()
    training_args = DPOTrainingConfig()
    training_args.eval_strategy = "epoch"  # Evaluates at end of each epoch
    training_args.load_best_model_at_end = False  # Keep latest model
    training_args.per_device_train_batch_size = 8
    training_args.gradient_accumulation_steps = 16
    # training_args.eval_steps = 250
    dataset_config = DatasetConfig(
        dataset = "tuluif",
        dataset_type = "pt",
        train_size = data,
    )
    sft_run_config = SFTRunConfig(
        dataset_config = DatasetConfig(
            dataset = "tuluif",
            dataset_type = "sft",
            train_size = checkpoint["data_points_seen"],
            dynamic_path = model_name
        ),
        model_name = MODEL,
        model_name_hf = HF_MODEL_MAP[MODEL], 
        task_name = "ifeval"
    )
    run_config = PTRunConfig(
        dataset_config = dataset_config,
        # model_name_hf = HF_MODEL_MAP[MODEL],  
        model_name = MODEL,  
        sft_run_config = sft_run_config,
        task_name = "ifeval",
        pft_method = "dpo",
        do_training = True
    )
    passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
        target_pass_at_k=[1.2],
        k_values=[1],
        n_samples=1,
        num_prompts=100,
        temperature=0.7,
        strict=False,
        enabled=True,
    )
    model, tokenizer, trainer = train_model_dpo(
        run_config = run_config,
        lora_config = lora_config,
        model_load_config = model_load_config,
        training_args = training_args,
        # passk_config = passk_config,
        # perplexity_thresholds= [0.1] # dummy value to periodically check perplexities too
    )
    del model, tokenizer, trainer
    gc.collect()
    torch.cuda.empty_cache()




In [ ]:
!nvidia-smi